In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
pip install patool

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import patoolib
patoolib.extract_archive('/content/train (2).rar',outdir='/content')

In [ ]:
import patoolib
patoolib.extract_archive('/content/test (1).rar',outdir='/content')

In [ ]:
import patoolib
patoolib.extract_archive('/content/gdrive/My Drive/trainecg.rar',outdir='/content')

In [ ]:
import glob
import scipy.misc
from numpy import genfromtxt
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow
from google.colab import files
from skimage.transform import resize
import os
from google.colab import drive, files
from io import BytesIO

if not os.path.exists('imgs4'):
  os.mkdir('imgs4')

n=0
for filepath in glob.iglob('/content/NORMAL/*.csv'):
#for filepath in glob.iglob('/content/drive/My Drive/P-108 s0013_rem.csv'):
    #print(filepath)
    img_array = genfromtxt(filepath, delimiter=',')
    #print(img_array.shape)
    #scipy.misc.toimage(image_array, cmin=0.0, cmax=1.0).save('outfile.jpg')
    im = Image.fromarray(img_array)
    image00= np.repeat(img_array[:, :, np.newaxis], 3, axis=2)
    #cv2_imshow(image)
    print(image00.shape)
    #plt.imshow(im)
    #new_image = im.resize((299, 299))
    #plt.imshow(new_image)
    image00 = resize(image00, (256, 256, 3))
    n=n+1
    print (n)
    #plt.imshow(image00)
    image00 = ((image00 * 255).astype(np.uint8))
    print(image00.size)
    print(image00.shape)
    # plt.imsave('output'+str(n),image00 , format='png' )
    image00 = Image.fromarray(image00.astype(np.uint8))
    image00.save("imgs4/check"+str(n)+".jpeg", "JPEG")



In [ ]:
# !rm -rf imgs
# !rm *.png
# !rm output*
!cp -r imgs4/* /content/drive/My\ Drive/travi4

In [ ]:
# !rm -rf imgs
# !rm *.png
# !rm output*
!cp -r imgs/* drive/My\ Drive/travi

In [ ]:
from PIL import Image, ImageDraw
from numpy import genfromtxt

g = open('/content/1.csv','r')
temp = genfromtxt(g, delimiter = ',')
im = Image.fromarray(temp).convert('RGB')
pix = im.load()
rows, cols = im.size
for x in range(cols):
  for y in range(rows):
    print (str(x) + " " + str(y))
    pix[x,y] = (int(temp[y,x] // 256 // 256 % 256),int(temp[y,x] // 256  % 256),int(temp[y,x] % 256))
im.save()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout,BatchNormalization,Activation,MaxPooling2D
from tensorflow.keras.models import Model,Sequential
from  tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
batch_size = 32


# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '/content/train',  # this is the target directory
        target_size=(256,256),  # all images will be resized to 512*512
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy

# print(train_generator.class_indices)
validation_generator = test_datagen.flow_from_directory(
        '/content/test',
        target_size=(256,256),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

In [ ]:
import keras
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout
from tensorflow.keras.layers import Input
input_tensor = Input(shape=(256, 256, 3))


# create the base pre-trained model
base_model =MobileNetV2(input_tensor=input_tensor,weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
#x=Dropout(0.5)


x = Dense(512, activation='relu')(x)
#x=Flatten()(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dropout(0.5)(x)

predictions = Dense(6, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 150 layers and unfreeze the rest:
for layer in model.layers[0:130]:
   layer.trainable = False
for layer in model.layers[130:]:
   layer.trainable = True
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
print("Number of layers in the base model: ", len(model.layers))

In [ ]:
def scheduler(epochs, lr):
  if epochs < 5:
    return lr
  else:
    return lr * tf.math.exp(-0.01)

In [ ]:
import tensorflow as tf

reduce_lr = keras.callbacks.ReduceLROnPlateau(
      monitor='val_loss',
        factor=0.2,
        patience=5,
        verbose=0,
        min_lr=0.01
    )
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1)
tensorboard=tf.keras.callbacks.TensorBoard(
    log_dir='logs', histogram_freq=0, write_graph=True, write_images=False,
    update_freq='epoch', profile_batch=2, embeddings_freq=0,
    embeddings_metadata=None
)
learning_scheduler=tf.keras.callbacks.LearningRateScheduler(
    scheduler, verbose=0
)
    # early stopping when `val_loss` stops improving


# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate

In [ ]:
model.summary()

In [ ]:
epochs=50
nb_train_samples=3435
nb_validation_samples=935

In [ ]:

history=model.fit(
    train_generator,
    steps_per_epoch=int(nb_train_samples / batch_size),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=int(nb_validation_samples / batch_size),
    callbacks=[reduce_lr,learning_scheduler])
    
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model.predict_generator(validation_generator, nb_validation_samples // batch_size+1,verbose=True)
y_pred = np.argmax(Y_pred, axis=1)
y_true = validation_generator.labels
#print(validation_generator.labels)
#print(y_pred)
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))
print('Classification Report')
target_names = ['ALMI','AMI','ASMI','IMI','INFERIOR','NORMAL']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

In [ ]:
tf.keras.utils.plot_model(model,'mobilenetv2.png')

In [ ]:
from sklearn.model_selection import train_test_split

X='/content/testecg'
X_train, X_test = train_test_split(X, test_size = 0.3, random_state = 0)
print(X_train)

In [ ]:
import cv2
im = cv2.imread('/content/test/travi/check.png')

print(type(im))
# <class 'numpy.ndarray'>

print(im.shape)
print(type(im.shape))

In [ ]:
model.save('MobileV2Net.h5')

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
fig, c_ax = plt.subplots(1,1, figsize = (12, 8))

def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)

    for (idx, c_label) in enumerate(target_names): # all_labels: no of the labels, for ex. 
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    #c_ax.plot(fpr, fpr, 'b-', label = 'Random Guessing')
        c_ax.plot(fpr, tpr, 'b-', linestyle='--',color='orange', label='ALMI')
        c_ax.plot(fpr, tpr, 'b-', linestyle='--',color='green', label='AMI')
        c_ax.plot(fpr, tpr, 'b-', linestyle='--',color='red', label='ASMI')
        c_ax.plot(fpr, tpr, 'b-', linestyle='--',color='pink', label='IMI')
        c_ax.plot(fpr, tpr, 'b-', linestyle='--',color='blue', label='INFERIOR')
        c_ax.plot(fpr, tpr, 'b-', linestyle='--',color='grey', label='NORMAL')

    return roc_auc_score(y_test, y_pred, average=average)

# calling
validation_generator.reset() # resetting generator
y_pred = model.predict_generator(validation_generator, verbose = True)
y_pred = np.argmax(y_pred, axis=1)
multiclass_roc_auc_score(validation_generator.classes, y_pred)

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import keras
import numpy as np
import os
import cv2

seed = 7
np.random.seed(seed)
data_x = []
data_y = []

path = '/content/train'
classname = 0
for folder in os.listdir(path):
    labels = [0. for _ in range(6)]
    labels = np.array(labels)
    for image in os.listdir(os.path.join(path, folder)):
        image_path = os.path.join(path, folder, image)
        img = keras.preprocessing.image.load_img(image_path, target_size=(256, 256))
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = img_array/255.0
        data_x.append(img_array)
        labels[classname] = 1.
        data_y.append(labels)
    classname += 1

data_x, data_y = np.array(data_x), np.array(data_y)
data_x, data_y = shuffle(data_x, data_y, random_state=seed)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
hists = []

for train, test in kfold.split(data_x, data_y):
    train_x = []
    train_y = []
    test_x = []
    test_y = []
    for x in train:
        train_x.append(data_x[x])
        train_y.append(data_y[x])
    for x in test:
        test_x.append(data_x[x])
        test_y.append(data_y[x])

    train_x, train_y, test_x, test_y = np.array(train_x), np.array(train_y), np.array(test_x), np.array(test_y)
    model = keras.models.load_model('/content/MobileV2Net.h5')
    model.compile(Adam(lr=0.0001), 'categorical_crossentropy',  metrics=['accuracy'])
    history = model.fit(train_x, train_y, validation_split=0.2, epochs=30, batch_size=32, verbose=2)
    hists.append(history)
    scores = model.evaluate(test_x, test_y, verbose = 2)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

model.save('kfold_mobilenet.h5')

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=len(hists), ncols=2, figsize=(20,20))

for i, history in enumerate(hists):
    history = history.history
    ax[i, 0].plot(history['accuracy'], label='Accuracy')
    ax[i, 0].plot(history['val_accuracy'], label='Validation Accuracy')
    ax[i, 0].legend(loc='upper left')
    # plt.xlabel('Epochs')
    # plt.ylabel('Accuracy')
    ax[i, 0].set_title('Accuracy')
    ax[i, 1].plot(history['loss'], label='Loss')
    ax[i, 1].plot(history['val_loss'], label='Validation Loss')
    ax[i, 1].legend(loc='upper left')
    # ax[i, 1].xlabel('Epochs')
    # ax[i, 1].ylabel('Loss')
    ax[i, 1].set_title('Loss')
    plt.xlabel('Epochs')

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt


fig, c_ax = plt.subplots(1,1, figsize = (12, 8))

def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)

    for (idx, c_label) in enumerate(target_names): # all_labels: no of the labels, for ex. 
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    c_ax.plot(fpr, fpr, 'b-', label = 'Random Guessing')
    c_ax.legend(loc='lower right')
    return roc_auc_score(y_test, y_pred, average=average)

import os
if os.path.exists('mobilenet.h5'):
    model = keras.models.load_model('mobilenet.h5')
# calling
validation_generator.reset() # resetting generator
y_pred = model.predict(validation_generator, verbose = True)
y_pred = np.argmax(y_pred, axis=1)
multiclass_roc_auc_score(validation_generator.classes, y_pred)